In [ ]:
!pip install ultralytics

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Open video file
video_path = "test.mp4"
cap = cv2.VideoCapture(video_path)

# Optional: Prepare output video writer
out = cv2.VideoWriter("output.avi",
                      cv2.VideoWriter_fourcc(*'XVID'),
                      30.0,
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 on the current frame
    results = model(frame)

    # Draw results on frame
    annotated_frame = results[0].plot()

    out.write(annotated_frame)

    # Press 'q' to exit early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model (trained on COCO)
model = YOLO("yolov8n.pt")

# Open video file
video_path = "test.mp4"
cap = cv2.VideoCapture(video_path)

# Optional: Prepare output video writer
out = cv2.VideoWriter("output.avi",
                      cv2.VideoWriter_fourcc(*'XVID'),
                      30.0,
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 on the current frame
    results = model(frame)[0]  # get first result (frame-level)

    # Filter for class ID 32 -> "sports ball" in COCO
    ball_boxes = results.boxes
    if ball_boxes is not None:
        for box, cls in zip(ball_boxes.xyxy, ball_boxes.cls):
            if int(cls) == 32:
                x1, y1, x2, y2 = map(int, box[:4])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                cv2.putText(frame, "BALL", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Write annotated frame
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()


In [ ]:
import cv2
from ultralytics import YOLO
from math import hypot
import pytesseract

# Load model
model = YOLO("yolov8n.pt")

# Open video
video_path = "test.mp4"
cap = cv2.VideoCapture(video_path)

# Output video writer
out = cv2.VideoWriter("output.avi",
                      cv2.VideoWriter_fourcc(*'XVID'),
                      30.0,
                      (int(cap.get(3)), int(cap.get(4))))

# Functions
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea + 1e-5)

def center(box):
    x1, y1, x2, y2 = box
    return ((x1 + x2) // 2, (y1 + y2) // 2)

def is_touch(player_box, ball_box, threshold=40):
    px, py = center(player_box)
    bx, by = center(ball_box)
    return hypot(px - bx, py - by) < threshold

def extract_jersey_number(frame, box):
    x1, y1, x2, y2 = box
    jersey_crop = frame[y1:y2, x1:x2]
    jersey_crop = cv2.resize(jersey_crop, (100, 150))  # Normalize size
    gray = cv2.cvtColor(jersey_crop, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, config='--psm 6 digits')
    return ''.join(filter(str.isdigit, text.strip()))

# Tracking variables
target_player_box = None
touches = 0
last_touched = False
salah_detected = False

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    current_players = []
    current_ball = None

    # Extract boxes
    for box, cls in zip(results.boxes.xyxy, results.boxes.cls):
        x1, y1, x2, y2 = map(int, box[:4])
        if int(cls) == 0:  # person
            current_players.append((x1, y1, x2, y2))
        elif int(cls) == 32:  # ball
            current_ball = (x1, y1, x2, y2)

    # First few frames: identify Salah by jersey number 11
    if not salah_detected and frame_idx < 50:
        for player_box in current_players:
            number = extract_jersey_number(frame, player_box)
            if number == "11":
                print("Salah detected with jersey #11!")
                target_player_box = player_box
                salah_detected = True
                break

    # Track Salah by IoU
    if salah_detected and target_player_box:
        best_iou = 0
        best_match = None
        for player in current_players:
            iou_val = iou(target_player_box, player)
            if iou_val > best_iou:
                best_iou = iou_val
                best_match = player
        if best_match:
            target_player_box = best_match
            cv2.rectangle(frame, (target_player_box[0], target_player_box[1]),
                          (target_player_box[2], target_player_box[3]), (0, 255, 0), 2)
            cv2.putText(frame, "Salah", (target_player_box[0], target_player_box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Draw ball
    if current_ball:
        cv2.rectangle(frame, (current_ball[0], current_ball[1]),
                      (current_ball[2], current_ball[3]), (0, 255, 255), 2)
        cv2.putText(frame, "Ball", (current_ball[0], current_ball[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Detect touch
    if target_player_box and current_ball:
        if is_touch(target_player_box, current_ball):
            if not last_touched:
                touches += 1
                print(f"Touch #{touches}")
            last_touched = True
        else:
            last_touched = False

    # Draw touch count
    cv2.putText(frame, f"Salah Touches: {touches}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    out.write(frame)

    frame_idx += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
0: 384x640 6 persons, 1 sports ball, 148.5ms
Speed: 3.1ms preprocess, 148.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 sports balls, 147.9ms
Speed: 3.1ms preprocess, 147.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 151.6ms
Speed: 4.7ms preprocess, 151.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 159.0ms
Speed: 3.3ms preprocess, 159.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 sports ball, 145.8ms
Speed: 3.4ms preprocess, 145.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 149.6ms
Speed: 3.5ms preprocess, 149.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 sports ball, 144.0ms
Speed: 3.6ms preprocess, 144.0ms infere

In [ ]:
# SYSTEM DEPENDENCIES
!apt update && apt install -y tesseract-ocr
# PYTHON DEPENDENCIES
!pip install pytesseract opencv-python-headless ultralytics deep_sort_realtime

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,710 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,343 kB]
Fetched 8,181 kB in 3s (2,900 kB/s)
Reading package lists... Done
Building dependency tree... Done
Readi

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import pytesseract
import numpy as np
from math import hypot

# Path to tesseract (for Colab)
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

# Load model
model = YOLO("yolov8n.pt")  # You can use yolov8s.pt or yolov8m.pt for better results
tracker = DeepSort(max_age=30)

# Load video
video_path = "/content/test.mp4"  # ← Upload your video here
cap = cv2.VideoCapture(video_path)

# Output
output_path = "/content/output.avi"
out = cv2.VideoWriter(output_path,
                      cv2.VideoWriter_fourcc(*'XVID'),
                      30.0,
                      (int(cap.get(3)), int(cap.get(4))))

# Utility functions
def extract_jersey_number(frame, box):
    x1, y1, x2, y2 = map(int, box)
    jersey_crop = frame[y1:y2, x1:x2]
    if jersey_crop.size == 0:
        return ""
    resized = cv2.resize(jersey_crop, (100, 150))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, config='--psm 6 digits')
    return ''.join(filter(str.isdigit, text.strip()))

def center(box):
    x1, y1, x2, y2 = box
    return ((x1 + x2) // 2, (y1 + y2) // 2)

def is_touch(player_box, ball_box, threshold=40):
    px, py = center(player_box)
    bx, by = center(ball_box)
    return hypot(px - bx, py - by) < threshold

# Tracking
target_jersey = "9"
target_id = None
touches = 0
last_touched = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    detections = []
    current_ball = None
    for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
        x1, y1, x2, y2 = map(int, box[:4])
        if int(cls) == 0:
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf.item(), 'player'))
        elif int(cls) == 32:
            current_ball = (x1, y1, x2, y2)

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = track.to_ltrb()
        x1, y1, x2, y2 = int(l), int(t), int(r), int(b)

        # If player ID not assigned yet, check OCR
        if target_id is None:
            number = extract_jersey_number(frame, (x1, y1, x2, y2))
            if number == target_jersey:
                target_id = track_id
                print(f"🎯 Found jersey #{target_jersey} as track ID {track_id}")

        # If it's our target player
        if track_id == target_id:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Player {target_jersey}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            if current_ball:
                if is_touch((x1, y1, x2, y2), current_ball):
                    if not last_touched:
                        touches += 1
                        print(f"⚽️ Touch #{touches}")
                    last_touched = True
                else:
                    last_touched = False

    # Draw touches
    cv2.putText(frame, f"Touches: {touches}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Done! Output video saved to:", output_path)


ModuleNotFoundError: No module named 'deep_sort_realtime'